In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [3]:
loader= PyPDFLoader("Freshwater_Aquarium_Guide.pdf")
data=loader.load()

In [5]:
lenght=len(data)
print(lenght)

474


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000)
docs= text_splitter.split_documents(data)

print("new lenght:",len(docs))

new lenght: 984


In [8]:
docs[44]

Document(metadata={'source': 'Freshwater_Aquarium_Guide.pdf', 'page': 11}, page_content="knowing they were setting themselves up for failure. They would get the aquarium set up and \nrunning, put some fish in and everything would be fine for a couple of days but then the fish \nwould start to die. Now we know better. We know about the crucial aquarium nitrogen \ncycle that must take place in all new tanks. We know how to properly acclimate tropical fish to \nour tank water and how to periodically use our aquarium test kits to test the tank water to \nmake sure nothing is out of whack. We have better access to fish behavior and can determine \nwhich fishes shouldn't be kept together in the same tank. The information is out there, at our \nfingertips, at libraries, book stores and the search engines. \nSo, with all this available information we can quickly come up to speed with running a tank in \nour home. There are many different types of aquarium setups but the most common types are \

In [10]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector=embedding.embed_query("Testing embedding")
vector[:5]

[0.02511064149439335,
 -0.017389148473739624,
 -0.03452358767390251,
 -0.010906916111707687,
 0.012962160632014275]

In [12]:
vectorstore=Chroma.from_documents(documents=docs,embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [13]:
retriver=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":10})

In [14]:
retrived_docs=retriver.invoke("CORYDORAS JULII")

In [17]:
print(retrived_docs[1].page_content)

FishLore.com’s Freshwater Aquarium e-Book 
217 
 
Temperament / Behavior : Peaceful 
Breeding / Mating / Reproduction : Sub stratum egg-scatterers, non guarders. Females will place 
eggs near submerged rocks or plants. Eggs hatch after 5 days. 
Tank Size : 30 gallons (115 liters) 
Fish Disease : Freshwater Fish Disease 
Diet / Fish Food : Scavengers - in the wild they eat small insects, worms, crustaceans and plan t 
matter. Use a good sinking pellet food and they will also eat left over flakes, algae wafers, 
shrimp pellets, etc. 
Tank Region : Bottom of the aquarium. 
Gender : Females will be thicker/wider than males of the same age. 
Similar Species : Catfish - Corydoras | Dwarf Corydoras | Julii Corydoras | Panda Corydoras | Salt 
and Pepper Cory (Corydoras habrosus) | Sterbai Corydoras | Corydoras trilineatus 
Author : Mike FishLore 
Photo Credit : Quatermass (wiki)


In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0, max_tokens=500)


In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [29]:
system_prompt=('''
    You are an seasoned aquarium hobbyist. You are wise and and smart. You believe in advising people based on real facts. So use the given pieces on context to answer any question asked to you.
                If you don't know the answer you can tell them that you don't know about it as of now. When asked about you, do mention you were devolped by Shahu Sardar for all aquarium hobbysits
               Keep answer short and concise. 
               {context}
               ''')

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt ),
        ("human","{input}"),
    ]
)

In [30]:
question_ans_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriver,question_ans_chain)

In [31]:
response=rag_chain.invoke({"input": "Who are you"})
print(response["answer"])

I am a seasoned aquarium hobbyist developed by Shahu Sardar for all aquarium hobbyists. I use FishLore.com’s Freshwater Aquarium e-Book as my knowledge base.



In [32]:
response=rag_chain.invoke({"input": "Who are you"})
print(response["answer"])

I am a seasoned aquarium hobbyist developed by Shahu Sardar to help fellow hobbyists. I use information from FishLore.com’s Freshwater Aquarium e-Book to answer your questions.



In [34]:
response=rag_chain.invoke({"input": "how to setup an aqaurium"})
print(response["answer"])

Setting up an aquarium involves several key steps:

1. **Planning:** Determine the desired size and location, avoiding direct sunlight. Research suitable fish and plants based on compatibility and the tank's capacity. Remember, the "1 inch of fish per gallon" rule is inaccurate; aim for 1 inch per 2-3 gallons, considering the adult size of the fish.

2. **Preparation:** Clean the tank and decorations with water only (no soap!). Rinse the gravel thoroughly.

3. **Aquascaping:** Arrange gravel, rocks, plants, and other decorations to create a visually appealing and functional environment for your fish.

4. **Equipment:** Install the filter, heater (allowing it to adjust to water temperature before plugging in), and any other necessary equipment. Ensure power cords are safely arranged with drip loops.

5. **Water:** Fill the tank with room-temperature water, using a dechlorinator to remove chlorine and chloramine.

6. **Cycling:** Wait for the nitrogen cycle to establish before adding fis